# surprise 라이브러리 설치

In [5]:
conda install -c conda-forge scikit-surprise

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\sehyu\anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-surprise-1.1.0      |py37hc8d92b1_1001         608 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         608 KB

The following NEW packages will be INSTALLED:

  scikit-surprise    conda-forge/win-64::scikit-surprise-1.1.0-py37hc8d92b1_1001




scikit-surprise-1.1. | 608 KB    |            |   0% 
scikit-surprise-1.1. | 608 KB    | ########9  |  89% 
scikit-surprise-1.1. | 608 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


# 패키지 불러오기

In [2]:
import sklearn
import surprise
import pandas as pd
import numpy as np

# 저장된 meetings와 moim 데이터 불러오기와서 합침

In [7]:
meetings = pd.read_csv('./meeting_data.csv')
moim = pd.read_csv('./moim_data.csv')
df = pd.concat([meetings, moim])
df

,meeting_id,uid,writer,created_at,updated_at,is_period,meeting_date,period_date,is_class,max_person,...,unit,is_active,like_cnt,view_cnt,score,main_category,tags,title,area1,area2
0,108923,https://www.frip.co.kr/hosts/617,그 아저씨,2020-05-30 20:58:48.363822,2020-05-30 20:58:48.363822,0,2020-05-30 20:58:48.363822,NaN,1,20,...,원,1,11222,0,4.77,0,NaN,[널위한선물] 도예 공방 원데이 / 도자기 그릇 만들기 (예약가능),서울시,성동구
1,133413,https://www.frip.co.kr/hosts/5545,MCJ,2020-05-30 20:58:52.561736,2020-05-30 20:58:52.561736,0,2020-06-03 19:30:00.000000,NaN,1,20,...,원,1,265,0,5.00,0,NaN,BYOB 각자 와인 1병씩 가져오는 MCJ 홈파티,서울시,강남구
2,101774,https://www.frip.co.kr/hosts/746,아이홉 맥주공방 (I'HOP),2020-05-30 20:58:55.192702,2020-05-30 20:58:55.192702,0,2020-05-31 13:00:00.000000,NaN,1,20,...,원,1,9848,0,4.76,0,NaN,[가정의달] 술술 놀면서 수제 맥주 만들기,서울시,송파구
3,133023,https://www.frip.co.kr/hosts/10654,비푸머스,2020-05-30 20:58:58.940685,2020-05-30 20:58:58.940685,0,2020-05-30 20:58:58.940685,NaN,1,20,...,원,1,888,0,4.81,0,NaN,[서울/홍대] 최상급 프랑스 향료로 나만의 향수 만들기 (예약 가능),서울시,마포구
4,115497,https://www.frip.co.kr/hosts/2839,지아네요리공작소,2020-05-30 20:59:02.833314,2020-05-30 20:59:02.833314,0,2020-05-30 20:59:02.833314,NaN,1,20,...,원,1,5674,0,4.89,0,NaN,[망원동] 스콘 2종 원데이클래스 (예약 가능),서울시,마포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,135609,https://www.frip.co.kr/hosts/11305,박초롱,2020-05-30 17:59:18.149290,2020-05-30 17:59:18.149290,0,2020-06-09 19:30:00.000000,NaN,1,20,...,원,1,66,0,0.00,0,NaN,[4회] 프로N잡러와 함께 - 언젠가 프리랜서,서울,마포구
158,134752,https://www.frip.co.kr/hosts/11160,아페리,2020-05-30 17:59:24.630295,2020-05-30 17:59:24.630295,0,2020-06-06 19:00:00.000000,NaN,1,20,...,원,1,118,0,5.00,0,NaN,2030 직장인 소셜 네트워킹 와인 파티,서울시,중구
159,133251,https://www.frip.co.kr/hosts/10766,수오재기,2020-05-30 17:59:31.681293,2020-05-30 17:59:31.681293,0,2020-06-16 00:00:00.000000,NaN,1,20,...,원,1,64,0,4.90,0,NaN,"15일 _ 토닥토닥, 나 칭찬하기",생활은,"일상에서,"
160,128055,https://www.frip.co.kr/hosts/5493,에스프레소,2020-05-30 17:59:38.988290,2020-05-30 17:59:38.988290,0,2020-06-05 19:45:00.000000,NaN,1,20,...,원,1,415,0,4.71,0,NaN,[여자 추가모집] 완벽한 타인,서울시,강남구


In [8]:
meeting_images = pd.read_csv('./meeting_images.csv')
moim_images = pd.read_csv('./moim_images.csv')
images = pd.concat([meeting_images, moim_images])
images

,Unnamed: 0,meeting_id,image_url
0,0.0,108923,None
1,1.0,133413,None
2,2.0,101774,None
3,3.0,133023,None
4,4.0,115497,None
...,...,...,...
157,NaN,135609,None
158,NaN,134752,None
159,NaN,133251,None
160,NaN,128055,None


# 합친 모임/강좌 정보와 이미지 링크 정보를 csv 파일로 저장

In [18]:
df.to_csv('meetings.csv', index=False, encoding='utf-8-sig')
images.to_csv('images.csv', index=False, encoding='utf-8-sig')

# 합침 정보 불러오기

In [3]:
df = pd.read_csv('meetings.csv')
images = pd.read_csv('images.csv')

In [25]:
df['content'][0]

'[내용][코로나19 관련 유의사항]1. 프립은 다수의 인원이 참여하는 프로그램이므로, 발열/호흡기 관련 증상, 감기 등의 질병이 발생한 대원님은 참여를 지양해주세요.2. 프립 참가 시에는 마스크 착용, 손 소독제 활용 등으로 안전에 특히 유의해주세요.3. 일정 변동 없이 진행되는 프립은 하단의 환불 규정을 따릅니다. 참여가 우려되시는 분들은 구매 시 신중한 선택 부탁드리며, 환불 규정을 숙지하여 기한 내 환불 신청 바랍니다.ㅡ프립에서 구매 후\xa0호스트 연락처를 문자로 보내드립니다.구매 전 문의사항은 Q&A 게시판을 이용해주세요.ㅡ도예 공방에서 세상에서 단 하나뿐인나만의 그릇 만들기음식 맛이 더 맛있어져요 :)그 아저씨 공방만의 특별 할인가더 많은 분들이 유익하고 즐거운 시간을 가지시길 바라며준비해 본 할인가 입니다 :D1인 이용권 35,000원 -> 32,000원1인 이용권 ★오전 클래스(10:30) 특별 할인★ 35,000원 -> 30,000원1인 이용권 ★10인 이상 단체 특별 할인★ 35,000원 -> 28,000원*10인 이상 단체 할인은 함께 참여하는 인원이 10명 이상 될 때 구매해주셔야 합니다!.\xa0.\xa0.우리에게 친숙한 소재인 흙을 통해 도예와 관련된 문화와 기술을 공유하며 다양한 작품 활동을 통해 폭넓은 창작의 세계를 경험하실 수 있습니다.-다양한 전시 및 경험을 갖춘 전문 강사진들과 함께 체계적이고 재미있는 수업이 진행되는 도예전문 공방입니다.^^-프립을 통해 만들어 볼 나만의 도예 작품은 자유주제로 자신이 원하는 아이템 하나를 선택한 후\xa0만들게 되고 핸드 빌딩 방식으로 제작됩니다.-핸드 빌딩 방식이란? 사람의 손으로 직접 모양을 만드는 제형 기법으로 만드는 사람 각각의 개성을 담을 수 있습니다.♥-실생활에도 유용하게 사용할 수 있는 나만의 예쁜 그릇! 함께 만들어 봐요. :D<아래사진은 공방에 실제로 배치된 샘플들입니다. 참고하세요 >아이템은 참고하시고 디자인은 본인에 손길에 따라 아이디어에 따라 달라집니다. 공방에는 사진

In [26]:
df['content'][1]

'[내용][코로나19 관련 유의사항]1. 프립은 다수의 인원이 참여하는 프로그램이므로, 발열/호흡기 관련 증상, 감기 등의 질병이 발생한 대원님은 참여를 지양해주세요.2. 프립 참가 시에는 마스크 착용, 손 소독제 활용 등으로 안전에 특히 유의해주세요.3. 일정 변동 없이 진행되는 프립은 하단의 환불 규정을 따릅니다. 참여가 우려되시는 분들은 구매 시 신중한 선택 부탁드리며, 환불 규정을 숙지하여 기한 내 환불 신청 바랍니다.안녕하세요. 30회 이상의 파티 플래닝/호스팅 경력을 가진 프립의 공동창업 멤버 MCJ입니다.파티 경력 외에도 프립 호스트 및 독서 모임, 영화 모임 플랫폼에서도 파트너 경험이 있습니다.파티계에는 이런 썰이 있다고 합니다. "믿고 가는 MCJ 파티!"집에서 소규모로 와인 모임을 하려고 와인잔도 구비하고, 의자도 8개나 구입했습니다 ㅎㅎ(심지어 와인셀러에 와인도 가득 채워두었다는!)그래서 이제는 실험 차원 + 재미로 프립에서 BYOB (Bring Your Own Bottle) 모임을 진행해보려고 합니다.가져오신 와인 리스트에 맞춰(페어링시켜서) 맛집 배달을 시킬 예정이고요와인 리스트가 좀 애매하다 싶으면 제 와인셀러에서도 와인을 꺼내올 예정입니다 ^^ (N빵)※ 와인잔 깨뜨리시면 Broken Charge 있으니 유의해주세요!(잘 깨지지 않는 튼튼한 잔인데.. 이미 하나 깨졌네요..뀨)이 기회로 좋은 분들 많이 알게 되면 좋겠습니다.신청자가 0명인 프립은 제가 추가로 개설해서 그렇습니다.보통 24시간 이내 마감되니, 자리가 있을 때 신청하세요 :)- 첫 프립 BYOB 와인모임 사진 @ 2020.05.10 일요일 -모임 후 다음날 마셨던 와인병 사진을 이렇게 찍어서 보내드리고 있습니다.2020.05.10 와인 리스트2020.05.15 와인 리스트2020.05.17 와인 리스트2020.05.20 와인 리스트2020.05.22 와인 리스트Tip #1와인 구입 전에 Vivino(비비노) 앱으로 라벨을 촬영하시면, 해당 와인의 평균 구입가는 얼마인지,

In [18]:
df.meeting_id = df.meeting_id.astype('int')

In [16]:
df = df.drop_duplicates(["meeting_id"])
df

,meeting_id,uid,writer,created_at,updated_at,is_period,meeting_date,period_date,is_class,max_person,...,unit,is_active,like_cnt,view_cnt,score,main_category,tags,title,area1,area2
0,108923,617,그 아저씨,2020-05-30 20:58:48.363822,2020-05-30 20:58:48.363822,0,2020-05-30 20:58:48.363822,NaN,1,20,...,원,1,11222,0,4.77,0,NaN,[널위한선물] 도예 공방 원데이 / 도자기 그릇 만들기 (예약가능),서울시,성동구
1,133413,5545,MCJ,2020-05-30 20:58:52.561736,2020-05-30 20:58:52.561736,0,2020-06-03 19:30:00.000000,NaN,1,20,...,원,1,265,0,5.00,0,NaN,BYOB 각자 와인 1병씩 가져오는 MCJ 홈파티,서울시,강남구
2,101774,746,아이홉 맥주공방 (I'HOP),2020-05-30 20:58:55.192702,2020-05-30 20:58:55.192702,0,2020-05-31 13:00:00.000000,NaN,1,20,...,원,1,9848,0,4.76,0,NaN,[가정의달] 술술 놀면서 수제 맥주 만들기,서울시,송파구
3,133023,10654,비푸머스,2020-05-30 20:58:58.940685,2020-05-30 20:58:58.940685,0,2020-05-30 20:58:58.940685,NaN,1,20,...,원,1,888,0,4.81,0,NaN,[서울/홍대] 최상급 프랑스 향료로 나만의 향수 만들기 (예약 가능),서울시,마포구
4,115497,2839,지아네요리공작소,2020-05-30 20:59:02.833314,2020-05-30 20:59:02.833314,0,2020-05-30 20:59:02.833314,NaN,1,20,...,원,1,5674,0,4.89,0,NaN,[망원동] 스콘 2종 원데이클래스 (예약 가능),서울시,마포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3563,135609,11305,박초롱,2020-05-30 17:46:04.470024,2020-05-30 17:46:04.470024,0,2020-06-09 19:30:00.000000,NaN,1,20,...,원,1,66,0,0.00,0,NaN,[4회] 프로N잡러와 함께 - 언젠가 프리랜서,서울,마포구
3564,134752,11160,아페리,2020-05-30 17:46:08.744022,2020-05-30 17:46:08.744022,0,2020-06-06 19:00:00.000000,NaN,1,20,...,원,1,118,0,5.00,0,NaN,2030 직장인 소셜 네트워킹 와인 파티,서울시,중구
3565,133251,10766,수오재기,2020-05-30 17:46:11.626022,2020-05-30 17:46:11.626022,0,2020-06-16 00:00:00.000000,NaN,1,20,...,원,1,64,0,4.90,0,NaN,"15일 _ 토닥토닥, 나 칭찬하기",생활은,"일상에서,"
3566,128055,5493,에스프레소,2020-05-30 17:46:15.305022,2020-05-30 17:46:15.305022,0,2020-06-05 19:45:00.000000,NaN,1,20,...,원,1,415,0,4.71,0,NaN,[여자 추가모집] 완벽한 타인,서울시,강남구


In [10]:
images = images.drop("Unnamed: 0", axis=1)

In [14]:
images = images.drop_duplicates(["meeting_id"])

In [15]:
images

,meeting_id,image_url
0,108923,None
1,133413,None
2,101774,None
3,133023,None
4,115497,None
...,...,...
3563,135609,None
3564,134752,None
3565,133251,https://res.cloudinary.com/frientrip/image/upl...
3566,128055,None


In [12]:
images.dtypes

Unnamed: 0    float64
meeting_id      int64
image_url      object
dtype: object